# GCAT FB15k-237

### Content

* I. Initial

* II. Load Corpus

* III. Train GAT

* IV. Train Conv

* V. Evaluation

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
rm -rf /content/*

# I. Initial

In [ ]:
%%bash
rm -rf GCAT
git clone https://github.com/hmthanh/GCAT.git

Cloning into 'GCAT'...
Checking out files: 100% (118/118), done.


In [ ]:
!yes | cp --i /content/GCAT/* /content/

cp: -r not specified; omitting directory '/content/GCAT/data'
cp: -r not specified; omitting directory '/content/GCAT/output'


In [ ]:
%%bash
rm -rf /content/data/
rm -rf /content/output/
rm -r /content/sample_data/

mv /content/GCAT/* /content/
rm -r /content/GCAT/

rm: cannot remove '/content/sample_data/': No such file or directory


Load Google Drive if you want to save model on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHJGNhG48-84Yh0XVrK18v5Uydu5ZGLKfzsIZ3QBSG1ZuxnEaM_Vr0
Mounted at /content/drive


## 1. Config

In [ ]:
#@title Cấu hình cài đặt quá trình huấn luyện { run: "auto" }

#@markdown Siêu tham số

dataset = "FB15k-237" #@param ["WN18RR", "WN18", "FB15k", "FB15k-237"] {allow-input: false}
cuda = True #@param {type: "boolean"}
save_gdrive = True #@param {type: "boolean"}
data_folder = "./data" #@param {type: "string"}
output_folder="./output" #@param {type: "string"}
drive_folder="/content/drive/My Drive" #@param {type: "string"}

epochs_gat = 3000 #@param {type: "number"}
epochs_conv = 150 #@param {type: "number"}
weight_decay_gat = float(1e-5) #@param {type:"raw"}
weight_decay_conv = float(1e-5) #@param {type:"raw"}
pretrained_emb = True #@param {type: "boolean"}
embedding_size = 50 #@param {type: "slider", min: 20, max: 200}
lr = float(1e-3) #@param {type:"raw"}
get_2hop = True #@param {type: "boolean"}
use_2hop = True #@param {type: "boolean"}
partial_2hop = True #@param {type: "boolean"}

#@markdown --- Tham số cho mô hình GAT
batch_size_gat = 272115 #@param {type: "number"}
valid_invalid_ratio_gat = 2 #@param {type: "slider", min: 1, max: 10}
drop_GAT = 0.3  #@param {type:"raw"}
alpha = 0.2  #@param {type:"raw"}
entity_out_dim = [100, 200]  #@param {type:"raw"}
nheads_GAT = [2, 2]  #@param {type:"raw"}
margin = 1 #@param {type: "slider", min: 1, max: 10}

#@markdown --- Tham số cho mô hình ConvKB
batch_size_conv = 128  #@param {type: "slider", min: 16, max: 512, step:16}
alpha_conv = 0.2  #@param {type:"raw"}
valid_invalid_ratio_conv = 40
out_channels = 50 #@param {type: "slider", min: 50, max: 1000, step:50}
drop_conv = 0.3  #@param {type:"raw"}

from create_config import Config

config = Config(dataset=dataset, data_folder=data_folder,cuda=cuda,output_folder=output_folder,save_gdrive=save_gdrive,drive_folder=drive_folder,epochs_gat=epochs_gat,epochs_conv=epochs_conv,weight_decay_gat=weight_decay_gat,weight_decay_conv=weight_decay_conv,pretrained_emb=pretrained_emb,embedding_size=embedding_size,lr=lr,get_2hop=get_2hop,use_2hop=use_2hop,partial_2hop=partial_2hop,batch_size_gat=batch_size_gat,valid_invalid_ratio_gat=valid_invalid_ratio_gat,drop_GAT=drop_GAT,alpha=alpha,entity_out_dim=entity_out_dim,nheads_GAT=nheads_GAT,margin=margin,batch_size_conv=batch_size_conv,alpha_conv=alpha_conv,valid_invalid_ratio_conv=valid_invalid_ratio_conv,out_channels=out_channels,drop_conv=drop_conv)

config.dumps_config()

config

dataset: FB15k-237
data_folder: ./data
output_folder: ./output
drive_folder: /content/drive/My Drive
save_gdrive: True
cuda: True
epochs_gat: 3000
epochs_conv: 150
weight_decay_gat: 1e-05
weight_decay_conv: 1e-05
pretrained_emb: True
embedding_size: 50
lr: 0.001
get_2hop: True
use_2hop: True
partial_2hop: True
batch_size_gat: 272115
valid_invalid_ratio_gat: 2
drop_GAT: 0.3
alpha: 0.2
entity_out_dim: [100, 200]
nheads_GAT: [2, 2]
margin: 1
batch_size_conv: 128
alpha_conv: 0.2
valid_invalid_ratio_conv: 40
out_channels: 50
drop_conv: 0.3

## 2. WN18RR

In [ ]:
#@title Cấu hình cài đặt WN18RR { run: "auto" }

#@markdown Siêu tham số

dataset = "WN18RR" #@param ["WN18RR", "WN18", "FB15k", "FB15k-237"] {allow-input: false}
cuda = True #@param {type: "boolean"}
data_folder = "./data"
output_folder="./output" #@param {type: "string"}
drive_folder="/content/drive/My Drive" #@param {type: "string"}
save_gdrive = True #@param {type: "boolean"}
epochs_gat = 3600 #@param {type: "number"}
epochs_conv = 200 #@param {type: "number"}
weight_decay_gat = float(5e-6) #@param {type:"raw"}
weight_decay_conv = float(1e-5) #@param {type:"raw"}
pretrained_emb = True #@param {type: "boolean"}
embedding_size = 50 #@param {type: "slider", min: 20, max: 200}
lr = float(1e-3) #@param {type:"raw"}
get_2hop = True #@param {type: "boolean"}
use_2hop = True #@param {type: "boolean"}
partial_2hop = False #@param {type: "boolean"}

#@markdown --- Tham số cho mô hình GAT
batch_size_gat = 86835 #@param {type: "number"}
valid_invalid_ratio_gat = 2 #@param {type: "slider", min: 1, max: 10}
drop_GAT = 0.3  #@param {type:"raw"}
alpha = 0.2  #@param {type:"raw"}
entity_out_dim = [100, 200]  #@param {type:"raw"}
nheads_GAT = [2, 2]  #@param {type:"raw"}
margin = 5 #@param {type: "slider", min: 1, max: 10}

#@markdown --- Tham số cho mô hình ConvKB
batch_size_conv = 128  #@param {type: "slider", min: 16, max: 512, step:16}
alpha_conv = 0.2  #@param {type:"raw"}
valid_invalid_ratio_conv = 40
out_channels = 500 #@param {type: "slider", min: 100, max: 1000, step:100}
drop_conv = 0.0  #@param {type:"raw"}

from create_config import Config

config = Config(dataset=dataset,data_folder=data_folder,cuda=cuda,output_folder=output_folder,save_gdrive=save_gdrive,drive_folder=drive_folder,epochs_gat=epochs_gat,epochs_conv=epochs_conv,weight_decay_gat=weight_decay_gat,weight_decay_conv=weight_decay_conv,pretrained_emb=pretrained_emb,embedding_size=embedding_size,lr=lr,get_2hop=get_2hop,use_2hop=use_2hop,partial_2hop=partial_2hop,batch_size_gat=batch_size_gat,valid_invalid_ratio_gat=valid_invalid_ratio_gat,drop_GAT=drop_GAT,alpha=alpha,entity_out_dim=entity_out_dim,nheads_GAT=nheads_GAT,margin=margin,batch_size_conv=batch_size_conv,alpha_conv=alpha_conv,valid_invalid_ratio_conv=valid_invalid_ratio_conv,out_channels=out_channels,drop_conv=drop_conv)

config.dumps_config()

config

dataset: WN18RR
data_folder: ./data
output_folder: ./output
drive_folder: /content/drive/My Drive
save_gdrive: True
cuda: True
epochs_gat: 3600
epochs_conv: 200
weight_decay_gat: 5e-06
weight_decay_conv: 1e-05
pretrained_emb: True
embedding_size: 50
lr: 0.001
get_2hop: True
use_2hop: True
partial_2hop: False
batch_size_gat: 86835
valid_invalid_ratio_gat: 2
drop_GAT: 0.3
alpha: 0.2
entity_out_dim: [100, 200]
nheads_GAT: [2, 2]
margin: 5
batch_size_conv: 128
alpha_conv: 0.2
valid_invalid_ratio_conv: 40
out_channels: 500
drop_conv: 0.0

Load config

# II. Create corpus


In [ ]:
!python 1_create_corpus.py

number of unique_entities -> 14505
number of unique_entities -> 9809
number of unique_entities -> 10348
test_triples (20466, 3)
test_adjacency_mat (3, 20466)
entity2id 14541
relation2id 237
headTailSelector 237
unique_entities_train (14505,) 

Initialised relations and entities from TransE
Graph created
length of graph keys is  13781
time taken  1331.1761288642883
length of neighbors dict is  13222
Total triples count 310116, training triples 272115, validation_triples 17535, test_triples 20466
tcmalloc: large alloc 1073741824 bytes == 0x575ae000 @  0x7f8a682841e7 0x5acbbb 0x4e05c6 0x4e2022 0x4e1fcb 0x4e248b 0x4e2e4d 0x4e2b00 0x4e2a04 0x4e0ea8 0x4e263b 0x4e34a6 0x5eb622 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x50b053 0x634dd2 0x634e87
1. Created Corpus Successfully !


## III. Training GAT encoder

In [ ]:
!python 2_training_encoder.py

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Epoch 1750 , average loss 0.00374743458814919 , epoch_time 1.771026611328125

epoch->  1751
Iteration-> 0  , Iteration_time-> 1.7781 , Iteration_loss 0.0036
Epoch 1751 , average loss 0.0036164899356663227 , epoch_time 1.778261423110962

epoch->  1752
Iteration-> 0  , Iteration_time-> 1.7901 , Iteration_loss 0.0036
Epoch 1752 , average loss 0.0035992730408906937 , epoch_time 1.7901809215545654

epoch->  1753
Iteration-> 0  , Iteration_time-> 1.7930 , Iteration_loss 0.0036
Epoch 1753 , average loss 0.0036121434532105923 , epoch_time 1.7931485176086426

epoch->  1754
Iteration-> 0  , Iteration_time-> 1.8214 , Iteration_loss 0.0037
Epoch 1754 , average loss 0.0037104999646544456 , epoch_time 1.8215806484222412

epoch->  1755
Iteration-> 0  , Iteration_time-> 1.8370 , Iteration_loss 0.0037
Epoch 1755 , average loss 0.003700513858348131 , epoch_time 1.8371012210845947

epoch->  1756
Iteration-> 0  , Iteration_time-> 1.8351 , Iteration_

cuda:0


# III Training conv decoder

In [ ]:
%%bash
python 3_training_decoder.py

# V. Evaluation

### 1. Evaluating

In [ ]:
!python 4_evalution.py

Loading corpus
Loading model
Sampled indices
test set length  20466
0
4. Evaluation Successfully !


### 2. Load result

In [ ]:
ab= {}
ab[1] = "Tjanh"
ab

{1: 'Tjanh'}

In [ ]:
relation2id = ["12", "323", "234"]
left_entity_avg = {}
for i in range(len(relation2id)):
    print(i, "dsf")
    print("left_entity_avg[i]", i, " i ", left_entity_avg[i])
    print("left_entity[i]", left_entity[i])

0 dsf


KeyError: ignored

In [ ]:
left_entity_avg[0] = "234"